In this notebook we explore the possibilities of fuzzy search algorithms in finding similarities.

**Classification using fuzzy matching**

-Classify whether question pairs are duplicate or not

-Let us start with importing the necessary modules for exploring the data.